## LSTM Text Classification with Keras

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30505 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 6)
[+] Number of entries in each category:
[+] Training:
 [ 7787.  7450.  4776. 10868.  1036.    83.]
[+] Validation:
 [1857. 1820. 1253. 2756.  287.   27.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


Load embedding matrix into an `Embedding` layer. Toggle `trainable=False` to prevent the weights from being updated during training.

In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

### Structure
[Reference](https://github.com/richliao/textClassifier), [LTSM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/)

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(14,dropout=0.3,recurrent_dropout=0.3))(embedded_sequences)
preds = Dense(6, activation='softmax')(l_lstm1)

In [4]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc'])

In [5]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=50, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoints", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [6]:
!rm -R logs

In [7]:
model.summary()
model.save('ltsm.h5')
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 30, 200)           6101200   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 28)                24080     
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 174       
Total params: 6,125,454
Trainable params: 24,254
Non-trainable params: 6,101,200
_________________________________________________________________
Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 23s 729us/step - loss: 1.3688 - acc: 0.4097 - val_loss: 1.2853 - val_acc: 0.4862
Epoch 2/200
32000/32000 [==========================

KeyboardInterrupt: 

In [ ]:
import pandas as pd
pd.DataFrame(model_log.history).to_csv("history-inception.csv")